In [21]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
import os
import copy
import glob
import numpy as np
import astropy.io.fits as fits
import matplotlib as mpl
import matplotlib.pyplot as plt

from astropy.table import Table
from astropy.modeling import models

from stingray.gti import create_gti_from_condition, gti_border_bins, time_intervals_from_gtis, cross_two_gtis
from stingray.utils import show_progress
from stingray.fourier import avg_cs_from_events, avg_pds_from_events, poisson_level, get_average_ctrate
from stingray import AveragedPowerspectrum, AveragedCrossspectrum, EventList
from stingray.modeling.parameterestimation import PSDLogLikelihood
from scipy.optimize import curve_fit
#from sklearn.mixture import GaussianMixturet
from sklearn import mixture
from sklearn.cluster import KMeans

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [22]:
import os
tmp='HE'
def powspec(evt,name):
    string="powspec cfile1='@%s' window='-' dtnb=1 nbint=256 nintfm=INDEF rebin=-1.03 plot=no outfile=%s_%s.fps norm=-2 clobber=YES" %(evt,name,tmp)
    print(string)
    os.system(string)

In [23]:
obsid='20201120'
fname="/Volumes/WD_BLACK/Data/1A0535p262/HXMT/date_combine/%s/lc_%s.list"%(obsid,tmp)
print(fname)
powspec(fname,obsid)

/Volumes/WD_BLACK/Data/1A0535p262/HXMT/date_combine/20201120/lc_HE.list
powspec cfile1='@/Volumes/WD_BLACK/Data/1A0535p262/HXMT/date_combine/20201120/lc_HE.list' window='-' dtnb=1 nbint=256 nintfm=INDEF rebin=-1.03 plot=no outfile=20201120_HE.fps norm=-2 clobber=YES
 
powspec 1.0 (xronos6.0)
 
 Series 1 file    1:/Volumes/WD_BLACK/Data/1A0535p262/HXMT/date_combine/20201120
 
 Selected FITS extensions: 1 - RATE TABLE;
 
 Source ............ 1A_0535+262         Start Time (d) .... 19173 02:26:50.684
 FITS Extension ....  1 - `COUNTS    `   Stop Time (d) ..... 19173 04:06:48.684
 No. of Rows .......          952        Bin Time (s) ......    1.000
 Right Ascension ... 84.7299             Internal time sys.. Converted to TJD
 Declination ....... 26.32               Experiment ........ HXMT     HE
 
 Corrections applied: Vignetting - No ; Deadtime - Yes; Bkgd - Yes; Clock - No
             values: 1.00000000       1.00000000       1.00000000
 
 Selected Columns:  1- Time;  2- Y-axis;  3- Y-

In [24]:
ff="%s_%s.fps"%(obsid, tmp)
hdu=fits.open(ff)
frequency=hdu[1].data.field('FREQUENCY')
xax_e=hdu[1].data.field('XAX_E')
power=hdu[1].data.field('POWER')
error=hdu[1].data.field('ERROR')
pds=np.c_[frequency-xax_e,frequency+xax_e,2*power*xax_e,2*error*xax_e]
np.savetxt('%s_%s.txt'%(obsid,tmp),pds,delimiter='\t')

In [25]:
def flx2xsp(obsid,tmp):
    file_tmp="%s_%s"%(obsid,tmp)
    string="flx2xsp %s.txt %s.pha %s.rsp"%(file_tmp,file_tmp,file_tmp)
    print(string)
    #os.system(string)
flx2xsp(obsid,tmp)

flx2xsp 20201120_HE.txt 20201120_HE.pha 20201120_HE.rsp
